# Sensors Analytics REST API

## Deploy the API
- Go to the *Machine* tab, then set *Incoming connections* to **ON**. The API will be accessible through the indicated tunnelling link.  
- Run the notebook.



## Import the required modules

In [1]:
import cherrypy
import redis
from datetime import datetime, timedelta

## Connect to the Redis Database

In [2]:
REDIS_HOST="redis-12999.c135.eu-central-1-1.ec2.redns.redis-cloud.com"
REDIS_PORT="12999"
REDIS_USERNAME='default'
REDIS_PASSWORD='LHz8VkDlHWjkAFh0jVh6WsyMQEe8c49D'

redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)

# Test Redis connection
try:
    response = redis_client.ping()
    print("Connected to Redis:", response)
except Exception as e:
    print("Redis connection failed:", e)
    exit(1)

Connected to Redis: True


## Instructions:
1) Create a class for each endpoint (status, sensors, sensor)
    - Status
    - Sensors
    - Sensor
2) For each endpoint, implement the required HTTP methods (GET, POST, PUT, DELETE)
    - Status: GET
    - Sensors: GET, POST
    - Sensor: GET, PUT, DELETE
3) Map each object to its target endpoint.
    - Status() -> "/status"
    - Sensors() -> "/sensors"
    - Sensor() -> "/sensor"

## Status Endpoint Class

In [3]:
class Status:
    exposed = True
    
    @cherrypy.tools.json_out()
    def GET(self):
        return {
            'redis_connected': bool(redis_client.ping()),
            'status': 'running'
        }

## Sensors Endpoint Class

In [4]:
class Sensors:
    exposed = True
    
    @cherrypy.tools.json_out()
    def GET(self):
        try:
            # Get all sensor entries from Redis
            sensors = []
            mac_address = None
            for key in redis_client.scan_iter("*"):
                if not mac_address:
                    mac_address = key.decode().split(":")[0]
                sensor = key.decode().split(":")[1]
                if mac_address not in sensors:
                    sensors.append(sensor)
            
            return {"sensors": sensors, "mac_address":  mac_address}
            
        except Exception as e:
            raise cherrypy.HTTPError(500, str(e))

    @cherrypy.tools.json_in()
    @cherrypy.tools.json_out()
    def POST(self):
        try:
            # Get JSON data from request body
            data = cherrypy.request.json
            
            # Check if mac_address is in the request
            if 'mac_address' not in data:
                raise cherrypy.HTTPError(400, 'Missing mac_address in request body')
            
            # get the mac_address from payload
            mac_address = data['mac_address']
            
            # Create temperature and humidity timeseries
            temp_key = f"{mac_address}:temperature"
            humid_key = f"{mac_address}:humidity"

            # handle different scenarios, 
            # for example in case database temp_key already exists but humid_key does not.
            # or vice versa
            if redis_client.exists(temp_key):
                if redis_client.exists(humid_key):
                    raise cherrypy.HTTPError(409, f'Sensor temperature, humidity both already exist.')
                else:
                    print(" humidity successfully registered")
                    redis_client.ts().create(humid_key)
                    return {"message": f"Sensor humidity successfully registered"}
                raise cherrypy.HTTPError(409, f'Sensor temperature already exists')
            else:
                redis_client.ts().create(temp_key)
                if redis_client.exists(humid_key):
                    raise cherrypy.HTTPError(409, f'Sensor humidity already exists')
                else:
                    redis_client.ts().create(humid_key)
                    return {"message": f"Sensor temperature, humidity successfully registered"}
            
        except cherrypy.HTTPError:
            raise
        except Exception as e:
            raise cherrypy.HTTPError(500, str(e))

## Sensor Endpoint Class

In [5]:
class HistoricalData:
    exposed = True
    
    def validate_date(self, date_str):
        try:
            return datetime.strptime(date_str, '%Y-%m-%d')
        except ValueError:
            raise cherrypy.HTTPError(400, f'Wrong format for date: {date_str}. Use YYYY-MM-DD format.')
    
    @cherrypy.tools.json_out()
    def GET(self, mac_address, start_date=None, end_date=None):
        try:
            # Debug logging
            cherrypy.log(f"Received request for mac_address: {mac_address}")
            cherrypy.log(f"start_date: {start_date}, end_date: {end_date}")

            # Validate required parameters
            if not mac_address:
                raise cherrypy.HTTPError(400, 'Missing MAC address')
            if not start_date:
                raise cherrypy.HTTPError(400, 'Missing start date')
            if not end_date:
                raise cherrypy.HTTPError(400, 'Missing end date')
                
            # Validate date formats and range
            start = self.validate_date(start_date)
            end = self.validate_date(end_date)
            cherrypy.log(f"Validated dates - start: {start}, end: {end}")
            
            if end <= start:
                raise cherrypy.HTTPError(400, 'End date must be greater than start date')
            
            # Convert dates to timestamps (milliseconds)
            start_ts = int(start.timestamp() * 1000)
            end_ts = int((end + timedelta(days=1)).timestamp() * 1000)
            cherrypy.log(f"Timestamp range: {start_ts} to {end_ts}")

            # Check if sensor exists
            temp_key = f"{mac_address}:temperature"
            humid_key = f"{mac_address}:humidity"
            
            cherrypy.log(f"Checking keys: {temp_key} and {humid_key}")
            cherrypy.log(f"Temperature key exists: {redis_client.exists(temp_key)}")
            cherrypy.log(f"Humidity key exists: {redis_client.exists(humid_key)}")

            if not (redis_client.exists(temp_key) and redis_client.exists(humid_key)):
                raise cherrypy.HTTPError(404, f'MAC address {mac_address} not found or data incomplete')

            try:
                # Fetch data from Redis TimeSeries
                temp_data = redis_client.ts().range(temp_key, start_ts, end_ts)
                humid_data = redis_client.ts().range(humid_key, start_ts, end_ts)
                cherrypy.log(f"Retrieved {len(temp_data)} temperature readings and {len(humid_data)} humidity readings")
            except redis.ResponseError as e:
                cherrypy.log(f"Redis error: {str(e)}")
                raise cherrypy.HTTPError(500, f'Redis TimeSeries error: {str(e)}')

            # Format response
            timestamps = []
            temperatures = []
            humidities = []
            
            # Process temperature and humidity data
            temp_dict = {ts: val for ts, val in temp_data}
            humid_dict = {ts: val for ts, val in humid_data}
            
            # Get all timestamps
            all_timestamps = sorted(set(temp_dict.keys()) | set(humid_dict.keys()))
            cherrypy.log(f"Total unique timestamps: {len(all_timestamps)}")
            
            for ts in all_timestamps:
                timestamps.append(ts)
                temperatures.append(temp_dict.get(ts, None))
                humidities.append(humid_dict.get(ts, None))
            
            return {
                "mac_address": mac_address,
                "timestamp": timestamps,
                "temperature": temperatures,
                "humidity": humidities
            }
                
        except cherrypy.HTTPError:
            raise
        except Exception as e:
            cherrypy.log(f"Unexpected error: {str(e)}")
            raise cherrypy.HTTPError(500, f'Internal server error: {str(e)}')

## Setup cherrypy and Map objects to their target endpoints

In [ ]:
# Global configuration
conf = {
    '/': {
        'request.dispatch': cherrypy.dispatch.MethodDispatcher(),
        'tools.sessions.on': True,
        'tools.response_headers.on': True,
        'tools.response_headers.headers': [('Content-Type', 'application/json')],
        'cors.expose.on': True
    }
}

# Mount endpoints
cherrypy.tree.mount(Status(), '/status', conf)
cherrypy.tree.mount(Sensors(), '/sensors', conf)
cherrypy.tree.mount(HistoricalData(), '/data', conf)

# Server configuration
cherrypy.config.update({'server.socket_host': '0.0.0.0'})
cherrypy.config.update({'server.socket_port': 8080})

# Start the server
cherrypy.engine.start()
cherrypy.engine.block()

[30/Jan/2025:14:58:40] ENGINE Bus STARTING
CherryPy Checker:
The config entry 'cors.expose.on' is invalid, because the 'cors' config namespace is unknown.
section: [/]

[30/Jan/2025:14:58:40] ENGINE Started monitor thread 'Autoreloader'.
[30/Jan/2025:14:58:40] ENGINE Serving on http://0.0.0.0:8080
[30/Jan/2025:14:58:40] ENGINE Bus STARTED
172.3.28.49 - - [30/Jan/2025:14:59:00] "GET /status HTTP/1.1" 200 46 "" "python-requests/2.32.3"
172.3.186.144 - - [30/Jan/2025:14:59:03] "POST /sensors HTTP/1.1" 409 1960 "" "python-requests/2.32.3"
172.3.28.49 - - [30/Jan/2025:14:59:16] "POST /sensors HTTP/1.1" 409 1960 "" "python-requests/2.32.3"
172.3.50.42 - - [30/Jan/2025:14:59:23] "POST /sensors HTTP/1.1" 409 1960 "" "python-requests/2.32.3"
[30/Jan/2025:14:59:27]  Received request for mac_address: 0xe45f01e89bd0
[30/Jan/2025:14:59:27]  start_date: 2025-01-23, end_date: 2025-01-30
[30/Jan/2025:14:59:27]  Validated dates - start: 2025-01-23 00:00:00, end: 2025-01-30 00:00:00
[30/Jan/2025:14:59:2

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5772165a-963f-4d38-9b9d-11ecac130c62' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>